In [1]:
!pip install txtai

Defaulting to user installation because normal site-packages is not writeable


In [2]:
! pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import glob
import re
#import pandas as pd
# import spacy
import txtai
#from IPython.display import HTML, display

from txtai.embeddings import Embeddings
from txtai.pipeline import Extractor

#pd.set_option("display.max_colwidth", None)
#import cv2
#import matplotlib.pyplot as plt
#import numpy as np

/home/jdownes/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, GenerationConfig

In [5]:
with open("sample.txt") as file:
    data = file.read().replace('\n',' ')
    # stupid sentence split
    data = data.split(". ")
     
print(data)

['Reston, VA, Thursday, Nov', "2, 2023, — MetroStar's Chief Executive Officer and Co-Founder, Ali Reza Manouchehri, was named a winner of the Greater Washington Government Contractor Awards unveiled on Nov", '1 at the 21st Annual Awards Show at the Ritz-Carlton Tysons Corner', ' Manouchehri earned the award of Executive of the Year in the $75 Million – $300 Million category on Wednesday night', 'Other notable winners included Carissa Christensen of BryceTech, named Executive of the Year in the up to $75 million category, and Carey Smith of Parsons, recognized as Executive of the Year in the over $300 million category', ' Presented by the Northern Virginia Chamber of Commerce and the Professional Services Council, the winners were announced at the evening gala, known as the “Academy Awards of Government Contracting.” In attendance, Manouchehri was humbled and honored to be presented with this award, acknowledging the exceptional talents of his fellow contenders — Sunny Singh from Aeyon,

In [6]:
'''
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large", model_max_length = 1000)
x = tokenizer(data)
print(x)
'''

'\ntokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large", model_max_length = 1000)\nx = tokenizer(data)\nprint(x)\n'

In [7]:
#https://github.com/MetroStar/maiden-ai/blob/main/QA-LLM.ipynb

embeddings = Embeddings(
    {"path": "sentence-transformers/multi-qa-mpnet-base-dot-v1", "content": True, "tokenize": True}
)
# Build embeddings index
#https://neuml.github.io/txtai/embeddings/
#simulate user based tags

'''
embeddings.index([{"text": "text to index", "flag": True,
                   "actiondate": "2022-01-01"}])
'''
num_users = 3
user_data=[]
row_id = 0
user_id = 0
for d in data:
    
    #user_data.append((row_id, d, {"user_id": user_id}))
    user_data.append({"text": d, "user_id":user_id})
    user_id +=1
    row_id +=1
    if user_id == num_users:
        user_id = 0 
print(user_data)
embeddings.index(user_data)

/home/jdownes/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


[{'text': 'Reston, VA, Thursday, Nov', 'user_id': 0}, {'text': "2, 2023, — MetroStar's Chief Executive Officer and Co-Founder, Ali Reza Manouchehri, was named a winner of the Greater Washington Government Contractor Awards unveiled on Nov", 'user_id': 1}, {'text': '1 at the 21st Annual Awards Show at the Ritz-Carlton Tysons Corner', 'user_id': 2}, {'text': ' Manouchehri earned the award of Executive of the Year in the $75 Million – $300 Million category on Wednesday night', 'user_id': 0}, {'text': 'Other notable winners included Carissa Christensen of BryceTech, named Executive of the Year in the up to $75 million category, and Carey Smith of Parsons, recognized as Executive of the Year in the over $300 million category', 'user_id': 1}, {'text': ' Presented by the Northern Virginia Chamber of Commerce and the Professional Services Council, the winners were announced at the evening gala, known as the “Academy Awards of Government Contracting.” In attendance, Manouchehri was humbled and 

In [8]:
results = embeddings.search("who won the executive of the year award")
print(results)

[{'id': '3', 'text': ' Manouchehri earned the award of Executive of the Year in the $75 Million – $300 Million category on Wednesday night', 'score': 0.7093396186828613}, {'id': '4', 'text': 'Other notable winners included Carissa Christensen of BryceTech, named Executive of the Year in the up to $75 million category, and Carey Smith of Parsons, recognized as Executive of the Year in the over $300 million category', 'score': 0.6936543583869934}, {'id': '6', 'text': '  The MetroStar team celebrating the Executive of the Year award', 'score': 0.640782356262207}]


In [9]:
#https://neuml.github.io/txtai/embeddings/query/

#query = "select object from txtai where similar(:x) AND user_id=(:y)"
#results = embeddings.search(query, parameters={"x": "who won the executive of the year award", "y":1})
query = "select * from txtai where similar('who won the executive of the year award') AND user_id=1"
results = embeddings.search(query)
print(results)

[{'indexid': 4, 'id': '4', 'text': 'Other notable winners included Carissa Christensen of BryceTech, named Executive of the Year in the up to $75 million category, and Carey Smith of Parsons, recognized as Executive of the Year in the over $300 million category', 'tags': None, 'entry': '2023-11-28 15:29:20.280804', 'data': '{"text": "Other notable winners included Carissa Christensen of BryceTech, named Executive of the Year in the up to $75 million category, and Carey Smith of Parsons, recognized as Executive of the Year in the over $300 million category", "user_id": 1}', 'object': None, 'score': 0.6936543583869934}, {'indexid': 1, 'id': '1', 'text': "2, 2023, — MetroStar's Chief Executive Officer and Co-Founder, Ali Reza Manouchehri, was named a winner of the Greater Washington Government Contractor Awards unveiled on Nov", 'tags': None, 'entry': '2023-11-28 15:29:20.280804', 'data': '{"text": "2, 2023, \\u2014 MetroStar\'s Chief Executive Officer and Co-Founder, Ali Reza Manouchehri

In [11]:
#TODO simulate chat conversation
shared_kb= [
    "the best restaurant in chicago is mcdonalds",
    "the best restaurant in phoenix is burger king"
]

user1_conversation =[
    "what are the good restaurants in chicago?"

]
user2_conversation =[
    "what are the good restaurants in phoenix?"
]

In [12]:
embeddings = Embeddings(
    {"path": "sentence-transformers/multi-qa-mpnet-base-dot-v1", "content": True, "tokenize": True}
)
# Build embeddings index
#https://neuml.github.io/txtai/embeddings/
#simulate user based tags

num_users = 3
chat_data=[]
row_id = 0
user_id = 0
for d in user1_conversation:
    chat_data.append({"text": d, "user_id":1})
for d in user2_conversation:
    chat_data.append({"text": d, "user_id":2})
for d in shared_kb:
    chat_data.append({"text": d, "user_id":0})
embeddings.index(chat_data)

In [13]:
query = "select * from txtai where similar('what city was i asking about restaurants in?') AND user_id=1 or user_id=0"
results = embeddings.search(query)
print(results)

query = "select * from txtai where similar('what city was i asking about restaurants in?') AND user_id=2 or user_id=0"
results = embeddings.search(query)
print(results)

[{'indexid': 0, 'id': '0', 'text': 'what are the good restaurants in chicago?', 'tags': None, 'entry': '2023-11-28 15:29:49.810986', 'data': '{"text": "what are the good restaurants in chicago?", "user_id": 1}', 'object': None, 'score': 0.5982734560966492}, {'indexid': 2, 'id': '2', 'text': 'the best restaurant in chicago is mcdonalds', 'tags': None, 'entry': '2023-11-28 15:29:49.810986', 'data': '{"text": "the best restaurant in chicago is mcdonalds", "user_id": 0}', 'object': None, 'score': 0.5662116408348083}, {'indexid': 3, 'id': '3', 'text': 'the best restaurant in phoenix is burger king', 'tags': None, 'entry': '2023-11-28 15:29:49.810986', 'data': '{"text": "the best restaurant in phoenix is burger king", "user_id": 0}', 'object': None, 'score': 0.522383451461792}]
[{'indexid': 2, 'id': '2', 'text': 'the best restaurant in chicago is mcdonalds', 'tags': None, 'entry': '2023-11-28 15:29:49.810986', 'data': '{"text": "the best restaurant in chicago is mcdonalds", "user_id": 0}', '

# LLM



In [14]:

tokenizer = AutoTokenizer.from_pretrained("amazon/MistralLite", model_max_length = 2000)
model = AutoModelForCausalLM.from_pretrained("amazon/MistralLite", pad_token_id = tokenizer.eos_token_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:50<00:00, 25.46s/it]


In [19]:
q = "When in chicago what is the best type of food to eat?"
mistral_q = f"<|prompter|>{q}</s><|assistant|>"

inputs = tokenizer(
    mistral_q,
    return_tensors="pt")

outputs = model.generate(
    **inputs, max_new_tokens=1000, use_cache=True, do_sample=True,
    temperature=0.2, top_p=0.95)

text = tokenizer.batch_decode(outputs)[0]


In [20]:
print(text)

<s><|prompter|> When in chicago what is the best type of food to eat?</s><|assistant|> Chicago is known for its diverse and delicious food scene, with something to suit every taste and budget. Here are some of the best types of food to try when in Chicago:

Deep-dish pizza: Chicago-style pizza is famous for its thick, buttery crust and generous toppings. Try a slice from Lou Malnati's, Giordano's, or Pizzeria Uno.

Chicago-style hot dogs: A classic Chicago food is the Chicago-style hot dog, served on a poppy seed bun with mustard, relish, onions, tomato wedges, pickle spear, sport peppers, and a dash of celery salt. Try one from Portillo's or Superdawg.

Italian beef sandwiches: Another Chicago classic is the Italian beef sandwich, made with thinly sliced roast beef, served on a crusty roll with sweet peppers and giardiniera (a mix of pickled vegetables). Try one from Al's Beef or Mr. Beef.

Chicago-style deep-dish pizza: Chicago-style pizza is famous for its thick, buttery crust and g

In [24]:
# add in contextual stuff
embeddings = Embeddings(
    {"path": "sentence-transformers/multi-qa-mpnet-base-dot-v1", "content": True, "tokenize": True}
)
chat_data=[]
user1_context=["I hate spicy food.", "My favorite type of foods are seafood."]
user2_context=["I love southern food", "I am open to trying all types of food"]
for d in user1_context:
    chat_data.append({"text": d, "user_id":1})
for d in user2_context:
    chat_data.append({"text": d, "user_id":2})
embeddings.index(chat_data)

In [25]:
user_id =1
query = f"select * from txtai where similar('{q}') AND user_id={user_id} or user_id=0"
results = embeddings.search(query)
print(results)


[{'indexid': 1, 'id': '1', 'text': 'My favorite type of foods are seafood.', 'tags': None, 'entry': '2023-11-28 16:07:23.142141', 'data': '{"text": "My favorite type of foods are seafood.", "user_id": 1}', 'object': None, 'score': 0.552369236946106}, {'indexid': 0, 'id': '0', 'text': 'I hate spicy food.', 'tags': None, 'entry': '2023-11-28 16:07:23.142141', 'data': '{"text": "I hate spicy food.", "user_id": 1}', 'object': None, 'score': 0.3844941258430481}]


In [29]:
fr = "".join([ r['text'] for r in results])
mistral_qc = f"<|prompter|>With the following additional context: {fr} Provide an answer to the following question: {q}</s><|assistant|>"
inputs = tokenizer(
    mistral_qc,
    return_tensors="pt")

outputs = model.generate(
    **inputs, max_new_tokens=1000, use_cache=True, do_sample=True,
    temperature=0.2, top_p=0.95)

text = tokenizer.batch_decode(outputs)[0]

In [30]:
print(text)

<s><|prompter|> With the following additional context: My favorite type of foods are seafood.I hate spicy food. Provide an answer to the following question: When in chicago what is the best type of food to eat?</s><|assistant|> As a foodie who loves seafood and hates spicy food, I would recommend trying out the following types of food when in Chicago:

1. Deep-dish pizza: Chicago is famous for its deep-dish pizza, which is a thick, doughy pizza with a layer of cheese and toppings on top. While it's not seafood, it's a must-try when in Chicago.

2. Chicago-style hot dogs: Another Chicago staple is the Chicago-style hot dog, which is served on a poppy seed bun with mustard, relish, onions, tomatoes, pickle spear, and celery salt.

3. Italian beef sandwich: This sandwich is made with thinly sliced roast beef, served on a long roll with au jus (beef gravy) and topped with peppers and giardiniera (a mix of pickled vegetables).

4. Fish fry: If you're looking for seafood, many restaurants in